# Klassifikation mit einem Neuronalen Netz

## Trainingsdaten

In [2]:
# %pip install torch
# %pip install torchvision
# %pip install matplotlib



Note: you may need to restart the kernel to use updated packages.


In [2]:
from torchvision import datasets, transforms
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

def load(data):
    return np.load(data)['arr_0']
# load dataset
train_data = load('Data\K49-data\k49-train-imgs.npz')
test_data = load('Data\K49-data\k49-test-imgs.npz')

train_labels = load('Data\K49-data\k49-train-labels.npz')
test_labels = load('Data\K49-data\k49-test-labels.npz')

In [4]:
train_data.data.shape, test_data.data.shape

((232365, 28, 28), (38547, 28, 28))

## Definition des Neuronalen Netzes

In [3]:
# set parameters for feed forward neural network
input_size = 28*28 #=784
hidden_size = 24010
num_classes = 49
num_epochs = 10
batch_size = 500
learning_rate = 0.001

#combine labels and data
train_data = train_data.astype(np.float32)  # Or any other appropriate data type

dataset = TensorDataset(torch.tensor(train_data), torch.tensor(train_labels))

# create dataloader

train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

# create neural network
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size).to(dtype=torch.float32) # Hidden layer
        self.relu = nn.ReLU() # Activation function
        self.l2 = nn.Linear(hidden_size, num_classes).to(dtype=torch.float32) # Output layer

    def forward(self, x):  # Forward pass
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes)


Wie erwartet sind die Gewichte des Neuronalen Netzes vor dem Training zufällig gewählt, so dass die Klassifikation misslingt.

## Training des Neuronalen Netz

In [11]:

# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (train_data, train_labels) in enumerate(train_loader):
        # reshape images to (batch_size, input_size)
        train_data = train_data.reshape(-1, 28*28)

        # forward pass
        outputs = model(train_data)
        loss = criterion(outputs, train_labels)

        # backward and optimize
        optimizer.zero_grad() # clear gradients
        loss.backward() # backpropagation
        optimizer.step() # update weights

        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch+1, num_epochs, i+1, total_step, loss.item()))



Epoch [1/10], Step [100/465], Loss: 0.8946
Epoch [1/10], Step [200/465], Loss: 0.7867
Epoch [1/10], Step [300/465], Loss: 0.7676
Epoch [1/10], Step [400/465], Loss: 0.6484
Epoch [2/10], Step [100/465], Loss: 0.5555
Epoch [2/10], Step [200/465], Loss: 0.4887
Epoch [2/10], Step [300/465], Loss: 0.4708
Epoch [2/10], Step [400/465], Loss: 0.6739
Epoch [3/10], Step [100/465], Loss: 0.5266
Epoch [3/10], Step [200/465], Loss: 0.4359
Epoch [3/10], Step [300/465], Loss: 0.5363
Epoch [3/10], Step [400/465], Loss: 0.4644
Epoch [4/10], Step [100/465], Loss: 0.3917
Epoch [4/10], Step [200/465], Loss: 0.5257
Epoch [4/10], Step [300/465], Loss: 0.5952
Epoch [4/10], Step [400/465], Loss: 0.4679
Epoch [5/10], Step [100/465], Loss: 0.5613
Epoch [5/10], Step [200/465], Loss: 0.4719
Epoch [5/10], Step [300/465], Loss: 0.5277
Epoch [5/10], Step [400/465], Loss: 0.7372
Epoch [6/10], Step [100/465], Loss: 0.4825
Epoch [6/10], Step [200/465], Loss: 0.3636
Epoch [6/10], Step [300/465], Loss: 0.5090
Epoch [6/10

## Test des Neuronalen Netzes

In [4]:
# test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on test images: {} %'.format(100 * correct / total))


KeyboardInterrupt: 

800min
hidden_size = 240100

num_classes = 49

num_epochs = 10

batch_size = 500

learning_rate = 0.001

Accuracy = 1.5587545456501624 %

47 min

hidden_size = 24010

num_classes = 49

num_epochs = 10

batch_size = 500

learning_rate = 0.001

Accuracy = 91.28913562713834 %

## Test mit kleinerem Trainingsdatensatz

In [5]:
# choose a small traings set randomly
total_images = len(train_data)
train_size =  int(0.25 * total_images)  # 70% der Bilder für das Training

train_data = load('Data\K49-data\k49-train-imgs.npz')
test_data = load('Data\K49-data\k49-test-imgs.npz')

train_labels = load('Data\K49-data\k49-train-labels.npz')
test_labels = load('Data\K49-data\k49-test-labels.npz')

train_data = train_data.astype(np.float32)  # Or any other appropriate data type
train_data_small = TensorDataset(torch.tensor(train_data[:train_size]), torch.tensor(train_labels[:train_size]))
test_data_small = TensorDataset(torch.tensor(test_data[train_size:]), torch.tensor(test_labels[train_size:]))


In [18]:
train_data, train_data_small

(array([[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]],
 
        [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         ...,
         [ 0.,  0.,  4., ..., 23.,  0.,  0.],
         [ 0.,  0.,  0., ...,  4.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]],
 
        [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]],
 
        ...,
 
        [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,

In [7]:
# train model_small on train_data_small
# print loss and accuracy on train_data_small and test_data_small

batch_size = 10
num_epochs = 10
learning_rate = 0.001
report = 5

model_small = NeuralNet(input_size, hidden_size, num_classes)



# create dataloader
train_loader_small = DataLoader(dataset=train_data_small, batch_size=batch_size, shuffle=True)
test_loader_small = DataLoader(dataset=test_data_small, batch_size=batch_size, shuffle=False)


# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_small.parameters(), lr=learning_rate)

# train the model
total_step = len(train_loader_small)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader_small):
        # reshape images to (batch_size, input_size)
        images = images.reshape(-1, 28*28)

        # forward pass
        outputs = model_small(images)
        loss = criterion(outputs, labels)

        # backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print loss on train_data_small and test_data_small
        if (i+1) % report == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

        # Print accuracy after processing all batches
        if (i+1) % report == 0:
        # Calculate accuracy on train_data_small
            with torch.no_grad():
                correct = 0
                total = 0
                for images, labels in train_loader_small:
                    images = images.reshape(-1, 28*28)
                    outputs = model_small(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

                print('Train Accuracy of the model on the train images: {} %'.format(100 * correct / total))
                print('Correct:', correct)
                print('Total:', total)

    # Calculate accuracy on test_data_small
            with torch.no_grad():
                correct = 0
                total = 0
                for images, labels in test_loader_small:
                    images = images.reshape(-1, 28*28)
                    outputs = model_small(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

                print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))
                print('Correct:', correct)
                print('Total:', total)

# ... (rest of your code)



# test the model
# In the test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))




Epoch [1/10], Step [5/5810], Loss: 679.5137
Train Accuracy of the model on the train images: 8.696700005164312 %
Correct: 5052
Total: 58091


ZeroDivisionError: division by zero

In [ ]:
# test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))


Test Accuracy of the model on the 10000 test images: 96.85 %
